## Toxicity Baseline NB
Competition location:  
https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification  


In [48]:
import numpy as np
import pandas as pd
import vocab as vocabulary
import collections
import utils

### Load data

In [2]:
train = pd.read_csv('/data/ToxicityData/train.csv')
test = pd.read_csv('/data/ToxicityData/test.csv')

In [3]:
test.head()

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


In [4]:
train.iloc[0]

id                                                                                 59848
target                                                                                 0
comment_text                           This is so cool. It's like, 'would you want yo...
severe_toxicity                                                                        0
obscene                                                                                0
identity_attack                                                                        0
insult                                                                                 0
threat                                                                                 0
asian                                                                                NaN
atheist                                                                              NaN
bisexual                                                                             NaN
black                

### Tokenization
This can be as simple as calling string.split() - good enough for English and many European languages - but we could also do something more sophisticated here. There are various types of tokenizers:  
  
1. nltk.tokenize.treebank import TreebankWordTokenizer
2. nltk.tokenize import WhitespaceTokenizer

In [5]:
from nltk.tokenize import WhitespaceTokenizer
white_token = WhitespaceTokenizer()

In [6]:
from keras.preprocessing.text import Tokenizer
keras_token = Tokenizer()

Using TensorFlow backend.


In [28]:
V = 30000
SEED = 23
VAL_SPLIT = 0.3

#### First, tokenize everything to build vocab
Only use vocabs from train data.

In [7]:
tokenize_all_one_list = white_token.tokenize(' '.join(train['comment_text'].tolist()))

In [8]:
len(set(tokenize_all_one_list))

1670966

There are 1.67 million tokens, do not have to use all of them as tokens

In [9]:
collections.Counter(tokenize_all_one_list).most_common(20)

[('the', 4261263),
 ('to', 2611234),
 ('and', 2096691),
 ('of', 2021781),
 ('a', 1880032),
 ('is', 1454734),
 ('in', 1294522),
 ('that', 1163635),
 ('for', 911179),
 ('I', 861783),
 ('you', 734810),
 ('are', 714218),
 ('be', 618319),
 ('not', 613791),
 ('have', 598834),
 ('it', 598509),
 ('on', 577471),
 ('with', 556536),
 ('as', 471924),
 ('they', 464629)]

In [10]:
collections.Counter(tokenize_all_one_list).most_common(V)[-1]

('scant', 141)

#### The 30kth token has 140 appearances, not too bad, we will use top 30k covab, and leave the rest as unknown
This step takes a long time.

In [12]:
vocab = vocabulary.Vocabulary(tokenize_all_one_list, size=30000)

### Are there words that are more particular to spams?




### Conversion to IDs
While there are a few ML models that operate directly on strings, in most cases (and always for neural networks) you'll need to convert the tokens to integer IDs that can index into a feature vector. To do this, we'll need to keep track of a vocabulary, which in its simplest form is just a dictionary.  

And unlike before, we are now tokenizing every row and then turn them into IDs using our vocab created


In [108]:
x_train = [vocab.words_to_ids(white_token.tokenize(train_row)) for train_row in train['comment_text'].tolist()]
x_test = [vocab.words_to_ids(white_token.tokenize(test_row)) for test_row in test['comment_text'].tolist()]

In [109]:
train['comment_text'].tolist()[0]

"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"

In [110]:
vocab.ids_to_words(x_train[0])

['This',
 'is',
 'so',
 'cool.',
 "It's",
 'like,',
 '<unk>',
 'you',
 'want',
 'your',
 'mother',
 'to',
 'read',
 '<unk>',
 'Really',
 'great',
 'idea,',
 'well',
 'done!']

In [111]:
train[train['comment_text'].str.contains("need one down under")]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
149299,424798,0.6,"giant hickeys. need one down under, hahaha",0.0,0.3,0.0,0.1,0.0,NaN,NaN,...,143325,approved,0,0,0,0,0,0.6,0,10


In [112]:
train.iloc[0,2]

"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"

### Create Sparse input matrix
For many language models, we need to convert inputs into a sparse matrix.   
For example, for simple Naive Bayes BOW, we need to convert each sentence into an array containing the entire vocabulary. Each sentence would only have few words out of the entire vocab, so we will end up with a very sparse matrix, with each sentence being a row, and each row has V entries corresponding to the vocabulary.   
  
We have a function in utils to convert sentences into sparse matrix. In this representation, instead of printing V for each row, we only print the words that have a count > 0. 

In [113]:
y_train = np.array(train['target'] > 0.5)
y_train[:5]

array([False, False, False, False,  True])

In [114]:
y_train = [1 if i else 0 for i in y_train]
y_train[:5]

[0, 0, 0, 0, 1]

In [116]:
for ind, y in enumerate(y_train):
    print(y)
    if y == 1:
        print(ind)
        print(vocab.ids_to_words(x_train[ind]))
        break

0
0
0
0
1
4
['haha', 'you', 'guys', 'are', 'a', 'bunch', 'of', 'losers.']


In [117]:
train[train['comment_text'].str.contains("you guys are a bunch of")]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [118]:
rng = np.random.RandomState(SEED)

indices = np.arange(len(x_train))

rng.shuffle(indices)  # in-place

x_train =  np.array(x_train)
y_train =  np.array(y_train)
# the indices slicing only works with np array

x_train_s = x_train[indices]
y_train_s = y_train[indices]


split_idx = int(VAL_SPLIT * len(x_train))
val_x = x_train_s[:split_idx]
val_y = y_train_s[:split_idx]

train_x = x_train_s[split_idx:]
train_y  = y_train_s[split_idx:]



In [123]:
for ind, y in enumerate(train_y):
    if y == 1:
        print(ind)
        print(vocab.ids_to_words(train_x[ind]))
        break

20
['Its', 'more', 'like', '<unk>', 'row,', 'i', 'would', 'hate', 'to', 'be', 'a', 'business', 'owner', 'on', 'the', 'mall', 'it', 'used', 'to', 'be', 'so', 'cool,', 'not', 'anymore', 'you', '<unk>', 'even', 'see', 'any', 'cops', 'walking', 'around,', 'pathetic.']


In [125]:
train[train['comment_text'].str.contains("i would hate to be a")]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
1522698,5984708,0.527027,"Its more like skid row, i would hate to be a b...",0.013514,0.013514,0.0,0.540541,0.0,NaN,NaN,...,379755,approved,0,0,0,2,0,0.0,0,74


In [122]:
for ind, y in enumerate(val_y):
    if y == 1:
        print(ind)
        print(vocab.ids_to_words(val_x[ind]))
        break

3
['Right!', 'And', 'our', 'federal', 'government', 'is', 'just', 'overrun', 'with', 'such', '<unk>', 'people', 'who', 'never', 'ever', '<unk>', 'When', 'more', 'than', 'half', 'of', 'US', 'States', 'recognize', 'the', 'medical', 'and', '<unk>', 'recreational', 'use', 'of', 'cannabis,', 'then', 'its', 'only', 'a', 'matter', 'of', 'time', 'before', 'the', 'criminal', 'abuse', 'of', 'US', 'citizens', 'will', 'end.', 'Cannabis', 'is', 'now', 'the', 'cash', 'cow', 'of', 'the', 'judicial', 'system', 'and', 'its', 'multiple', 'private', 'contractors,', 'such', 'as', 'prison', '<unk>', '<unk>', 'outdated', 'information', 'that', 'a', 'minority', 'hopes', 'that', 'with', 'repetition', 'will', 'come', 'true,', 'is', 'a', '<unk>', 'game.', '<unk>', 'in', 'the', '<unk>', 'year', 'old', '<unk>', '<unk>', 'propaganda', 'ignores', 'the', 'reality', 'of', 'the', 'benefits', 'and', 'safety', 'of', 'cannabis', 'that', 'has', 'been', 'proven', 'scientifically', 'over', 'and', 'over', 'again.', 'Even', '

In [124]:
train[train['comment_text'].str.contains("And our federal government is just")]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
414450,750418,0.833333,Right! And our federal government is just ove...,0.0,0.0,0.166667,0.833333,0.0,NaN,NaN,...,157210,approved,0,0,0,0,0,0.0,0,6


In [ ]:
train_x_sb = utils.id_lists_to_sparse_bow(train_x, V)
val_x_sb = utils.id_lists_to_sparse_bow(val_x, V)

In [ ]:
print(train_x_sb[0])

In [ ]:
print("Training set: x = {:s} sparse, y = {:s}".format(str(train_x_sb.shape), str(len(train_x))))
print("Test set:     x = {:s} sparse, y = {:s}".format(str(val_x_sb.shape),  str(len(val_x))))

## Naive Bayes
NB is used for classification, we we are going to turn the target variable into binary variable. 
This is only for testing purpose. For final model we do need a predicted probability so NB is out of the question.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

nb = MultinomialNB()

nb.fit(train_x_sb, train_y)
y_pred_val = nb.predict(val_x_sb)


acc = accuracy_score(val_y, y_pred_val)
print("Accuracy on test set: {:.02%}".format(acc))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

precision, recall, fscore, support = score(val_y, y_pred_val)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
total = 0
for ind, val in enumerate(y_pred_val):
    if val != val_y[ind]:
        print(vocab.ids_to_words(val_x[ind]))
        print("pred", val, "actual", val_y[ind])
        total += 1
    if total > 5:
        break
    
    

In [51]:
train[train['comment_text'].str.contains("A very slippery slope here")]

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
107928,374148,0.0,A very slippery slope here.\n\n-- The populari...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,140595,approved,0,0,0,0,0,0.0,0,4
523812,884689,0.0,WOW!! A very slippery slope here. Equating hum...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,163562,approved,0,0,0,5,7,0.0,0,4


### Canonicalization
Depending on the application, we might want to do some pre-processing to remove spurious variation in the text. For example, we might want to lowercase words to avoid storing separate features for "I" and "i", and we might want to replace numbers with a special token rather than keep track of every possible value.

utils have a basic transformation in utils.canonicalize_word. It's important to write different one for different tasks since the use of language can be quite different